# Preparing Data for Retrieval Augmented Generation

## Preparing Data Pulled from Wikipedia and Storing it in LanceDB

In this Python notebook, we will explore the process of preparing data that has been pulled from Wikipedia and storing it in LanceDB for Retrival Augmented Generation. Wikipedia is a vast source of information, and extracting relevant data from it can be a valuable task for various data tasks.

Additionally, we can leverage the extracted data to augment a large language model using the OpenAI API. By feeding the data into the language model, we can enhance its capabilities and generate more accurate and contextually relevant responses. This can be particularly useful in natural language processing tasks, chatbots, and text generation applications.

To use the OpenAI API for data augmentation, we need to ensure that the extracted data is properly formatted. We can then make API calls to the OpenAI service, passing in the formatted data as input and receiving augmented responses as output. This iterative process of training and fine-tuning the language model with the extracted data can significantly improve its performance and generate more coherent and context-aware text.

By combining the power of LanceDB for data storage and retrieval and the OpenAI API for language model augmentation, we can create sophisticated and intelligent systems that can process and generate text with enhanced accuracy and relevance.


## Import Libraries

In [1]:
pip install -q --upgrade python-dotenv unidecode openai numpy pandas torch bs4 requests lancedb tiktoken

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import openai

import numpy as np
import pandas as pd
import torch

# Used in cleaning the data 
from unidecode import unidecode 
from dateutil import parser

# To Load environment variables
from dotenv import load_dotenv, find_dotenv
import os

# Pulling data from wikipedia
from bs4 import BeautifulSoup
import requests
import time
import random

# vector database
from lancedb.pydantic import vector, LanceModel
import lancedb

# Counting tokens in prompt
import tiktoken

d:\GitHub\Retrieval-Augmented-Generation\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
## Define parameters

In [5]:
# get api key
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ.get("OPENAI_API_KEY")

# set model
config = {
    'llm_model' : "gpt-3.5-turbo-0301",
    'embedding_model' : "text-embedding-3-small",
    'years' : [2022, 2023, 2024],
    'databaseName' : "events",
    "max_token_count": 2048
}


## Pulling Data from Wikipedia

In [6]:
base_endpoint = 'https://en.wikipedia.org/w/api.php'

params = {
    "action": "query",
    "prop" : "extracts",
    "titles" : "2022",#"water|fire|earth",
    "formatversion": "2",
    "explaintext": "1",
    "exsectionformat": "wiki",
    "format": "json",
    # "exintro": "1"
}

# multiple titles seems to fail as im requesting the entire page
# returns multiple pages if exintro is set to 1
# https://www.mediawiki.org/wiki/Topic:Shyz4nnfs8ucpioz

In [7]:
titles = []

for year in config['years']:
    time.sleep(random.randint(0, 7))
    temp = requests.get(f"https://en.wikipedia.org/wiki/Special:AllPages?from={year}_in_&to=&namespace=0")
    # Parse the HTML content
    soup = BeautifulSoup(temp.content, 'html.parser')

    # Find all the <a> tags
    links = soup.find_all('a')

    # Extract the href attribute from each <a> tag
    link_urls = [link.get('href') for link in links]

    # Filter out None values and remove duplicates
    link_urls = list(filter(lambda x: x is not None, link_urls))
    link_urls = [link for link in link_urls if f'wiki/{year}' in link]
    link_urls = list(set(link_urls))

    # Filter out the links that contain %!
    link_urls = [link for link in link_urls if '%' not in link]

    # Add to the list of all links
    titles.extend([link.replace(f'/wiki/', '') for link in link_urls])

# Print the extracted links
print(titles)

titles.extend(config['years'])

['2022_in_Fiji', '2022_in_Slovenia', '2022_in_politics', '2022_in_Oman', '2022_in_Chile', '2022_in_Costa_Rica', '2022_in_Eagle_Fighting_Championship', '2022_in_Vietnam', '2022_in_Tamil_television', '2022_in_United_States_rugby_league', '2022_in_Ecuador', '2022_in_South_Korean_television', '2022_in_Belgium', '2022_in_Mauritania', '2022_in_Europe', '2022_in_Canada', '2022_in_Weather', '2022_in_Uganda', '2022_in_American_music', '2022_in_Thailand', '2022_in_South_Korean_music', '2022_in_Canadian_music', '2022_in_Australian_literature', '2022_in_Cyprus', '2022_in_Azerbaijan', '2022_in_Dutch_television', '2022_in_Irish_television', '2022_in_Palau', '2022_in_Saint_Vincent_and_the_Grenadines', '2022_in_Japanese_music', '2022_in_jazz', '2022_in_NASCAR', '2022_in_biotechnology', '2022_in_Bulgaria', '2022_in_Peru', '2022_in_paleoentomology', '2022_in_Slovakia', '2022_in_Somalia', '2022_in_Kazakhstan', '2022_in_Spanish_television', '2022_in_Transnistria', '2022_in_Qatar', '2022_in_Samoa', '2022_i

In [8]:
print("Number of titles: ", len(titles))

Number of titles:  1026


I will use the wikipedia-api to pull data from Wikipedia. Given a list of topics, I will only look at a few of the topics and pull the data from Wikipedia. I will then store the data in LanceDB for retrieval and augmentation.

In [9]:
events = []
fails = []
for title in ["2022", '2023', '2024', '2022_in_music', '2023_in_music',
     '2024_in_music', '2022_in_science', '2023_in_science', '2024_in_science',
      "2022_in_sports", "2023_in_sports", "2024_in_sports"]:


    time.sleep(random.randint(0, 15))
    # pull the page
    params["titles"] = title
    response = requests.get(base_endpoint, params=params)
    response_dict =  response.json()

    # extract the text
    try:
        page = response_dict['query']['pages'][0]['extract'].split('\n')
    except:
        fails.append(title)
        continue

    
    # remove intro
    page = page[2:]

    # remove unicode
    page = list(map(lambda x : unidecode(x), page))
    # remove empty strings
    page = list(filter(lambda x : len(x) > 0, page))
    # add to the list
    events.extend(list(map(lambda x : x + f" {title.replace('_', ' ')}", page)))
        
# convert to a dataframe
df = pd.DataFrame(events)
df.columns = ['text']

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13975 entries, 0 to 13974
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    13975 non-null  object
dtypes: object(1)
memory usage: 109.3+ KB


In [11]:
df.head()

,text
0,2022 was also dominated by wars and armed conf...
1,== Events == 2022
2,=== January === 2022
3,January 1 - The Regional Comprehensive Econom...
4,January 2 - Abdalla Hamdok resigns as Prime Mi...


## Cleaning and Preprocessing Data

In [12]:
# reomove extra spaces
df.loc['text'] = df['text'].str.strip()

# remove missing values
df = df.dropna()

# drop duplicates
df = df.drop_duplicates()

# remove headings and empty strings
df = df[(df['text'] != '') & (~df["text"].str.startswith("==")) ]


prefix = ""

for (i, row) in df.iterrows():
    if " - " not in row["text"]:
            try:
                # If the row's text is a date, set it as the new prefix
                parse(row["text"])
                prefix = row["text"]
            except:
                pass

    if ":" in row["text"]:
            
            temp = row["text"].split(":")[0]

            if " - " in temp:
                try:
                    parse(temp.split("-")[0])
                    prefix = temp
                except:
                    pass

            else:
                try:
                    # If the row's text is a date, set it as the new prefix
                    parse(temp)
                    prefix = temp
                except:
                    pass

    try:
        # if the row's text starts with a date
        parse(row["text"].split(" ")[0])
    except:         
        if prefix != "" :
            row["text"] = prefix + " - " + row["text"]
    else:
        pass

In [13]:
# save the data
df.to_csv('data/events.csv', index=False)

# look at the data
df.head()

,text
0,2022 was also dominated by wars and armed conf...
3,January 1 - The Regional Comprehensive Econom...
4,January 2 - Abdalla Hamdok resigns as Prime Mi...
5,January 4 - The five permanent members of the ...
6,January 5 - A nationwide state of emergency is...


## Get Embeddings for Data

In [14]:
from typing import Union, List, Dict

In [15]:
def get_embedding(text: Union[str,List[str]], model:str = "text-embedding-3-small")->List[float]:
    """ Get the embedding for a given text using the OpenAI API
    
    Args:
        text (Union[str,List[str]]): The input text or list of texts to get the embedding for.
        model (str, optional): The name of the model to use for embedding. Defaults to "text-embedding-3-small".
    
    Returns:
        List[float]: The embedding vector for the input text.
    """
    text = text.replace("\n", " ")

    embedding = openai.embeddings.create(
        input = text,
        model = model
    )

    return embedding.data[0].embedding

In [16]:
# This may take a few minutes
df["embeddings"] = df['text'].apply( lambda x: get_embedding(x, model=config['embedding_model']))
df.to_csv("data/events_with_embeddings.csv", index=False)

In [17]:
df.head()

,text,embeddings
0,2022 was also dominated by wars and armed conf...,"[-0.05098605528473854, 0.01658623293042183, 0...."
3,January 1 - The Regional Comprehensive Econom...,"[-0.0530412532389164, 0.024792009964585304, 0...."
4,January 2 - Abdalla Hamdok resigns as Prime Mi...,"[0.02006388083100319, -0.025998268276453018, 0..."
5,January 4 - The five permanent members of the ...,"[-0.0011166088515892625, 0.004961762577295303,..."
6,January 5 - A nationwide state of emergency is...,"[-0.0538049079477787, -0.0076721422374248505, ..."


## Store Data in LanceDB

In [18]:
# Defing schema for the database
class Events(LanceModel):
    text : str
    embeddings : vector(len(df.embeddings[0]))

# Connect to the database
db = lancedb.connect(config['databaseName'])

table_name = "events"

# Create or load the table
try:
    table = db.create_table(table_name, schema = Events)
except:
    table = db.open_table(table_name)

# Add the data to the database
table.add(df[['text','embeddings']])



## Creating Prompt for OpenAI API

In [20]:
# Get encoder
encoding = tiktoken.encoding_for_model(config['llm_model'])

# Get tokenizer
tokenizer = tiktoken.get_encoding(encoding.name)

In [21]:
def get_rows_sorted_by_relevance(question, df, limit=None) :
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, model=config['embedding_model'])
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    if limit:
        result =  table.search(np.array(question_embeddings),vector_column_name="embeddings").limit(limit).to_pandas()
    else:
        # returns 10 by default
        result =  table.search(np.array(question_embeddings),vector_column_name="embeddings").to_pandas()
    

    return result

In [22]:
def create_prompt(question: str, df: pd.DataFrame , max_token_count: int  ,limit: int =None) -> Dict[str, str] :
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    context = ""

    prompt = {
        "system": (f"You are a helpful assistant."
                " If you don't know the answer, you can say 'I don't know' or 'I'm not sure'."),
             

    }

    prompt_template = ("Context:\n {} \n\n###\n\n"
                        "Answer the question based on the context provided. \n" 
                        "Question: {}")


    current_token_count = len(tokenizer.encode(prompt_template)) + len(tokenizer.encode(prompt["system"])) + len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df, limit = limit)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    prompt["user"] = prompt_template.format("\n".join(context), question)
    return prompt
    

In [23]:
def format_prompt(prompt: dict) -> List[Dict]:
    """
    Given a prompt dictionary, return a string that can be sent to a
    Completion model
    
    Args:
        prompt (Dict): A dictionary containing the system and user prompts
    
    Returns:
        List[Dict]: A list of dictionaries containing the role and content of each prompt
    """

    return [{"role": "system", "content": prompt["system"]},{"role": "user", "content": prompt["user"]}]

## Questions

### Question 1 

In [24]:
question = "When is World Yo-Yo Contest?"
prompt = create_prompt(question, df, config['max_token_count'], limit=20)
print(prompt["user"])

Context:
 July 31 - August 3: 2024 World Yo-Yo Contest in  Cleveland 2024 in sports
October 13 - 17: 2022 IFBB World Fitness Championships in  Yeongju 2022 in sports
June 18 - 22: 2022 World Foil Championships 2022 in sports
January 15-20: 2023 Contender World Championship in  Perth 2023 in sports
April 29 - May 6: 2022 Optimist Asian & Oceanian American Championship in  Yeosu 2022 in sports
2023 World University Combat World Cup (2nd) 2023 in sports
TBD: 2022 World Dodgeball Championships 2022 in sports
May 21 - 26: World Cup #2 in  Yecheon 2024 in sports
August 6-10: Amateur Disc Golf World Championships in  Grand Rapids 2024 in sports
July 22-30: 2023 Mirror Class World Championship in  Rosses Point 2023 in sports
January 7 - 23: 2022 World Indoor Bowls Championship in  Great Yarmouth 2022 in sports
November 5 & 6: 2022 Trout Area Fishing World Championship 2022 in sports
World & Continental ChampionshipsSeptember 21-24: 2024 WWA Wakeboard & Wake Park World Championships in  Gold Co

In [25]:
response = openai.chat.completions.create(
  model = config['llm_model'],
  messages=format_prompt(prompt),
)
response.choices[0].message.content

'The World Yo-Yo Contest is scheduled from July 31 to August 3 in 2024 and will be held in Cleveland.'

### Question 2

In [28]:
question = "Was there a Chinese spy balloon?"
prompt = create_prompt(question, df, config['max_token_count'],limit=20)

response = openai.chat.completions.create(
  model = config['llm_model'],
  messages=[
    {"role": "system", "content": prompt["system"]},
    {"role": "user", "content": prompt["user"]},
  ]
)
response.choices[0].message.content

'Yes, the context states that there were alleged Chinese spy balloons being tracked by the US over the Americas.'